# Расширение выборки
В качестве данных для расширения выборки были выбраны другие образовательные направления тех же университетов (в силу того, что для них уже выработана технология поиска данных). Для московских университетов практически нигде не удалось найти комплексные базы образовательных документов.

* АлтГТУ: https://www.altstu.ru/structure/unit/ur/, https://www.altstu.ru/sveden/common/ -- смотреть по кафедрам
<!-- * АмГУ: https://www.amursu.ru/sveden/education/educationDocs/ -- на 07.07.2022 недоступно -->
* АлтГУ: https://www.asu.ru/sveden/education/programs/#bakalavriat
<!-- * АГУ: https://adygnet.ru/obrazovanie/opop-vo/ -->
* ИМСИТ: https://www.imsit.ru/sveden/education/programs/nachalynoe-obrazovanie-44.03.01.html, https://www.imsit.ru/sveden/education/eduaccred/, https://www.imsit.ru/sveden/education/eduop/
* ВолГУ: https://volsu.ru/struct/institutes/, https://volsu.ru/sveden/education/pages/education_programs.php, https://volsu.ru/sveden/education/pages/documents.php, https://volsu.ru/Abitur/bachelor/perechen-programm.php
<!-- * ВУиТ: http://ved.vuit.ru/Plans/Default.aspx, http://vuit.ru/faculties/, http://ved.vuit.ru/Dek/Default.aspx?mode=spets, http://vuit.ru/sveden/education/index.html#opop -->
<!-- * АГТУ: https://www.astu.org/sveden/education -->
<!-- * ВСЭИ:   http://www.vsei.ru/gum-napravleniya-podgotovki -->
<!-- * БашГУ:   https://bashedu.ru/sveden/, https://bashedu.ru/sveden/education/eduaccred/, https://bashedu.ru/sveden/education/eduop/ -->
<!-- * БГУ Петровского:   https://brgu.ru/sveden/education/ -- раздел "реализуемые ОП" -->
<!-- * БелГУ:   https://dekanat.bsu.edu.ru/blocks/bsu_nabor/nabor.php, http://abitur.bsu.edu.ru/abitur/sp_napr/03/ -->
<!-- * БГТУ:   https://www.tu-bryansk.ru/sveden/education/eduop/ -->
<!-- * ВоГУ:   https://vogu35.ru/sveden/education -->
<!-- * ВолгГТУ:   https://www.vstu.ru/sveden/education/, https://www.vstu.ru/sveden/education/ -->

In [1]:
import io
import re
import os, wget
import requests
import glob
from tqdm.notebook import tqdm

CURRPATH = os.getcwd()
DATAPATH = '\\'.join(CURRPATH.split('\\')[:-1]) + '\\new-data'

# Парсинг HTML-выкладок и скачивание документов
Заранее были созданы текстовые файлы с html-скриптом, содержащим в себе ссылки на документы аннотаций и названия дисциплин. Эти данные и выделяются далее.

In [2]:
regex_altgu_href = r'/sveden/education/programs/abstract/\d*?/">'
regex_altgu_name = r'<td>[^`]*?</td>'
altgu_predomain = 'https://www.asu.ru/'

regex_altgtu_href = r'<a href="/omkoorder/f\d{5}[.]pdf" class="new-files">'
regex_altgtu_name = r'class="new-files">[^`]*?</a><small>'
regex_altgtu = r'<a href="/omkoorder/f\d{5}[.]pdf" class="new-files">[^`]*?</a><small>'
altgtu_predomain = 'https://www.altstu.ru/'

regex_imsit_href = r'href="[^`]*?"'
regex_imsit_name = r'rel="noopener" title="[^`]*?">'
imsit_predomain = 'https://www.imsit.ru/'

regex_volgu_href = r'href="[^`]*?"'
regex_volgu_name = r'name=[^`]*?"'
volgu_predomain = 'https://volsu.ru/sveden/education/pages/'

In [3]:
altgu_href_offsets = [0, -3]
altgtu_href_offsets = [9, -20]
imsit_href_offsets = [6, -1]
volgu_href_offsets = [6, -1]

altgu_name_offsets = [4, -5]
altgtu_name_offsets = [18, -11]
imsit_name_offsets = [22, -2]
volgu_name_offsets = [5, -1]

In [4]:
# with io.open(DATAPATH + '\\АлтГТУ\\цифровая экономика.txt', encoding='utf-16') as f:
#     contents = f.read()
# contents = re.sub(r' {2,}', '', contents.replace('\n', ''))
# contents[:500]

In [5]:
# one = re.findall(regex_altgtu_href, contents)
# two = re.findall(regex_altgtu_name, contents)
# assert len(one) == len(two)
# len(one), len(two)

In [6]:
# two

Следующие функции могут стать основой для какого-нибудь парсинг-фреймворка.

In [7]:
def download_uno(url, local_filename, alert=False):
    
    try:
        wget.download(href, local_filename)
        return 'success'
    except:
        try:
            r = requests.get(url)
            f = open(local_filename, 'wb')
            for chunk in r.iter_content(chunk_size=512 * 1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
            f.close()
            return 'success'
        except Exception as e:
            if alert:
                print(e, 'for', url)
            return 'fail'
        
        
def relocate(file):
    splitted = file.split('\\')
    to = '\\'.join(splitted[:-1]) + '\\done\\'
    
    try:
        os.makedirs(to)
    except:
        pass
    
    to += splitted[-1]
    os.replace(file, to)
    


def download_files(filepath: str, predomain: str,
                   regex_href: str, regex_name: str,
                   href_offsets: list[int], name_offsets: list[int],
                   save_to: str = None, encoding='utf-16',
                   alert=False, alert_download_errors=True,
                   regs_sub: list = None):
    
    if save_to is None:
        save_to = filepath[:-3]
    try:
        os.makedirs(save_to)
    except:
        pass
    
    with io.open(filepath, encoding=encoding) as f:
        contents = f.read()
    contents = re.sub(r' {2,}', '', contents.replace('\n', '').replace('и?', 'й'))
    contents = re.sub(r'<!--<a [^`]*?<!--</a>-->', '', contents)
    
    hrefs = re.findall(regex_href, contents)
    names = re.findall(regex_name, contents)
    assert len(hrefs) == len(names)
    
    for i in range(len(hrefs)):
        href = predomain + hrefs[i][href_offsets[0]:href_offsets[1]]
        href = href.replace(' ', '%20').replace('&amp;', '&')
        
        name = names[i].lower()[name_offsets[0]:name_offsets[1]]
        if regs_sub is not None:
            for reg in regs_sub:
                name = re.sub(reg, '', name)
        name = name.lstrip().rstrip()
        
        if alert:
            print(name, ':', href)
            
        _ = download_uno(href, save_to + '\\' + name + '.pdf', alert=alert_download_errors)
        assert _ == 'success'
        
REGS_SUB = [
    r'аннотация',
    r'[":/,_()]',
    r'б1[.][воб0][.]\d\d',
    r'б1[.][во][.]д[эв][.]\d\d[.]\d\d\d?',
    r'фтд[.]?в?[.]\d\d',
    r'б1[.]o[.]27',
    r'\d\d[.]\d\d[.]\d\d',
    r'[.]дэ[.]\d\d[.]\d\d',
    r'[.]\d\d',
]

## АлтГТУ

In [8]:
for file in tqdm(glob.glob(DATAPATH + '\\АлтГТУ\\*.txt')):
    print(file.split('\\')[-1])
    download_files(file, altgtu_predomain,
              regex_altgtu_href, regex_altgtu_name,
              altgtu_href_offsets, altgtu_name_offsets,
              regs_sub=REGS_SUB)
    relocate(file)

  0%|          | 0/8 [00:00<?, ?it/s]

графический дизайн.txt
производственный менеджмент.txt
управление инновационными проектами.txt
управление малым бизнесом.txt
физико-химическое материаловедение.txt
цифровая экономика.txt
цифровые финансы.txt
электроэнергетические комплексы и системы.txt


## ВолГУ

In [9]:
for file in tqdm(glob.glob(DATAPATH + '\\ВолГУ\\*.txt')):
    print(file.split('\\')[-1])
    download_files(file, volgu_predomain,
              regex_volgu_href, regex_volgu_name,
              volgu_href_offsets, volgu_name_offsets,
              regs_sub=REGS_SUB)
    relocate(file)

  0%|          | 0/22 [00:00<?, ?it/s]

биоразнообразие и биоресурсы.txt
журналистика в системе современных медиакоммуникаций.txt
инженерное дело в живых системах.txt
мировые политические процессы 3++.txt
муниципальное управление 3++.txt
отечественная и всеобщая история археология.txt
отечественная филология.txt
педагогическое русский язык и литература.txt
перевод и переводоведение 3++.txt
практическая психология управления и психологическое консультирование 3++.txt
прикладные математика и физика.txt
радиотехника.txt
радиофизика.txt
реклама и связи с общественностью в коммерческой сфере.txt
сети связи и системы коммутации.txt
современная российская политика 3++.txt
социальная теория и прикладное социальное знание 3++.txt
управление данными и интеллектуальные технологии в цифровой экономике.txt
управление инновационным предприятием.txt
экономика и бизнес-планирование 3++.txt
экономическая статистика и анализ данных.txt
юриспруденция.txt


## ИМСИТ

In [10]:
for file in tqdm(glob.glob(DATAPATH + '\\ИМСИТ\\*.txt')):
    print(file.split('\\')[-1])
    download_files(file, imsit_predomain,
              regex_imsit_href, regex_imsit_name,
              imsit_href_offsets, imsit_name_offsets,
              encoding='ansi', regs_sub=REGS_SUB)
    relocate(file)

  0%|          | 0/14 [00:00<?, ?it/s]

антикризисное управление 3++.txt
гостиничная деятельность 3++.txt
государственная и муниципальная служба 3++.txt
графический дизайн 3++.txt
дизайн среды.txt
международный менеджмент.txt
педагогическое образование англ нем 3++.txt
педагогическое образование изобразительное искусство и информатика.txt
педагогическое образование история и право 3++.txt
прикладная информатика.txt
программная инженерия.txt
разработка и управление информационными системами 3++.txt
реклама и связи с общественностью 3++.txt
экономика финансы и учёт 3++.txt


## АлтГУ

In [11]:
for file in tqdm(glob.glob(DATAPATH + '\\АлтГУ\\*.txt')):
    print(file.split('\\')[-1])
    try:
        download_files(file, altgu_predomain,
                  regex_altgu_href, regex_altgu_name,
                  altgu_href_offsets, altgu_name_offsets,
                  regs_sub=REGS_SUB)
        relocate(file)
    except:
        print()
        print('BAD FOR', file)
        print()
        continue

  0%|          | 0/27 [00:00<?, ?it/s]

бизнес-информатика.txt
биология.txt
биотехнология.txt
гостиничное дело.txt
интеллектуальный анализ данных.txt
история россии и всеобщая история.txt
компьютерная электроника и телекоммуникации.txt
конфликт-менеджмент.txt
медиакоммуникации.txt
международные отношения.txt
менеджмент.txt
отечественная филология.txt
политические теории процессы и технологии.txt
политология.txt
психология.txt
радиофизика.txt
радиофизические технологии в цифровой экономике.txt
реклама и связи с общественностью.txt
русская филология.txt
социология цифрового общества.txt
социология.txt
теоретическая и экспериментальная химия.txt
универсальная журналистика.txt
физика.txt
химия.txt
юриспруденция общеправовой.txt
юриспруденция.txt
